In [1]:
import os
import copy
import folium
import flickrapi
import pandas as pd
import urllib.request
import geopandas as gpd
import datetime

# MAPEX Map creation

In [2]:
# Set up Flickr API

# Set up API key and secret
# not on GitHub

# Set up API
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

### Get Metadata

In [71]:
# Set up API
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

# Define search parameters
extras = 'url_o, geo, license, views, machine_tags, date_taken, date_upload, url_m, url_t, url_n'

# Define bounding box coordinates
bbox = '12.6159174934222271,47.0015568061715712,12.7230649836587197,47.0486549491658010'
per_page = 250
pages = 12

full_dict_list = []
for page in range(pages):
    photos = flickr.photos.search(bbox=bbox, extras=extras, per_page=per_page, page=page)
    
    photo_dict = photos['photos']['photo']
    
    full_dict_list.extend(photo_dict)

full_dict = {elem['id']: elem for elem in full_dict_list}
df = pd.DataFrame(full_dict).T
df = df[~(df.url_o.isna())]

df.to_csv("data/metadata_new.csv")

### Or Load

In [3]:
df = pd.read_csv("data/metadata_new.csv")

In [4]:
df = df.drop_duplicates(subset='title').sample(100)

In [5]:
df.columns

Index(['Unnamed: 0', 'id', 'owner', 'secret', 'server', 'farm', 'title',
       'ispublic', 'isfriend', 'isfamily', 'license', 'dateupload',
       'datetaken', 'datetakengranularity', 'datetakenunknown', 'views',
       'machine_tags', 'latitude', 'longitude', 'accuracy', 'context',
       'place_id', 'woeid', 'geo_is_public', 'geo_is_contact', 'geo_is_friend',
       'geo_is_family', 'url_o', 'height_o', 'width_o', 'url_m', 'height_m',
       'width_m', 'url_t', 'height_t', 'width_t', 'url_n', 'height_n',
       'width_n'],
      dtype='object')

### Create data for JS

In [6]:
wanted = ['id', 'owner', 'datetaken', 'dateupload', 'latitude', 'longitude', 'url_o', 'url_m', 'url_n', \
          'title', 'height_m', 'width_m', 'height_n', 'width_n']
df = df[wanted]

### Convert for UNIX timestamp

In [7]:
df.dateupload = pd.to_datetime(df.dateupload, unit='s').dt.date.astype(str)
df.datetaken = pd.to_datetime(df.datetaken).dt.date.astype(str)

In [8]:
st = ''
def stringbuild(row):
    s = ''
    
    s += '{'
    
    idx = row.index.tolist()
    
    for i, title in enumerate(idx):
        
        s += title
        s += ': '
        
        if type(row[i]) == str:
            s += "'"
            s += str(row[i])
            s += "'"
            
        else:
            s += str(row[i])
        s += ', '
    
    s = s[:-2]
    s += '},\n'
    
    return s


strings = df.apply(stringbuild, axis=1)

for s in strings:
    st += s

In [9]:
print(st[:-2])

{id: 4346687925, owner: '68222897@N00', datetaken: '2009-09-30', dateupload: '2010-02-10', latitude: 47.033836, longitude: 12.717063, url_o: 'https://live.staticflickr.com/4069/4346687925_ec2e6e6f0d_o.jpg', url_m: 'https://live.staticflickr.com/4069/4346687925_04e18fcac2.jpg', url_n: 'https://live.staticflickr.com/4069/4346687925_04e18fcac2_n.jpg', title: 'DSC_0300', height_m: 335, width_m: 500, height_n: 214, width_n: 320},
{id: 52700206442, owner: '32368927@N02', datetaken: '2022-10-11', dateupload: '2023-02-20', latitude: 47.001852, longitude: 12.643911, url_o: 'https://live.staticflickr.com/65535/52700206442_52b2ef5ff4_o.jpg', url_m: 'https://live.staticflickr.com/65535/52700206442_c011dbd50f.jpg', url_n: 'https://live.staticflickr.com/65535/52700206442_c011dbd50f_n.jpg', title: 'Fading Light in the Austrian Alps', height_m: 333, width_m: 500, height_n: 213, width_n: 320},
{id: 5491963157, owner: '79669681@N00', datetaken: '2011-02-18', dateupload: '2011-03-02', latitude: 47.010847